# OR case 1

In [111]:
import pandas as pd
import openpyxl
from gurobipy import *
import matplotlib.pyplot as plt
import numpy as np

In [95]:
# 如果你要用 plotly 畫甘特圖，可以用 Jupyter notebook 跑，用 Jupyter lab 會跑不出來
# 有任何問題歡迎寄信詢問助教 b07705036@ntu.edu.tw
import plotly.express as px

# display settings
from IPython.display import display
pd.options.display.max_columns = None
pd.options.display.max_rows = None
pd.set_option('display.float_format', lambda x: '%.4f' % x)

# 第一題的甘特圖
def gantt_plot_1(x_opt, P, y_opt, instance_no, open_hour=7, open_min=30):
    import datetime

    I = len(x_opt)
    M = len(y_opt[0])
    schedule_df = pd.DataFrame()
    open_time = datetime.datetime(2000, 1, 1, hour=open_hour, minute=open_min)
    schedule_df['START_TIME'] = [(open_time + datetime.timedelta(hours=x_opt[i]) - datetime.timedelta(hours=P[i])) for i in range(I)]
    schedule_df['END_TIME'] = [(open_time + datetime.timedelta(hours=x_opt[i])) for i in range(I)]

    machine_ID = []
    for i in range(I):
        for m in range(M):
            if y_opt[i][m] == 1:
                machine_ID.append(str(m + 1))
    schedule_df['Machine_ID'] = machine_ID

    schedule_df['Job_ID'] = [str(i + 1) for i in range(I)]
    color_list = ["#C0392B", "#EC7063", "#1F618D", "#34495E", "#3498DB", "#1E8449", "#28B463", "#F1C40F", "#F39C12", "#A6ACAF", "#7D3C98", "#9B59B6", "#2C3E50", "#EF553B", "#636EFA", "#BDC3C7", "#F39C12"]
    fig = px.timeline(schedule_df, x_start="START_TIME", x_end="END_TIME", y="Machine_ID", color="Job_ID",
                      color_discrete_sequence=color_list[0:I],
                      category_orders={"Job_ID": [str(i+1) for i in range(I)], "Machine_ID": [str(m+1) for m in range(M)]},
                      title="Instance " + str(instance_no) + " Problem 1 Gantt Chart")
    fig.update_xaxes(tickformat="%H:%M")
    fig.update_layout(legend_title="Lot ID")
    fig.show()
    fig.write_image("Instance " + str(instance_no) + " Problem 1.pdf")

# 第二、三題的甘特圖
def gantt_plot_2_3(x_opt, P, y_opt, instance_no, problem_no, open_hour=7, open_min=30):
    import datetime

    I = len(x_opt)
    S = len(x_opt[0])
    M = len(y_opt[0][0])
    schedule_df = pd.DataFrame()
    open_time = datetime.datetime(2000, 1, 1, hour=open_hour, minute=open_min)
    schedule_df['START_TIME'] = [(open_time + datetime.timedelta(hours=x_opt[i][s]) - datetime.timedelta(hours=P[i][s])) for i in range(I) for s in range(S)]
    schedule_df['END_TIME'] = [(open_time + datetime.timedelta(hours=x_opt[i][s])) for i in range(I) for s in range(S)]

    machine_ID = []
    for i in range(I):
        for s in range(S):
            for m in range(M):
                if y_opt[i][s][m] == 1:
                    machine_ID.append(str(m + 1))
    schedule_df['Machine_ID'] = machine_ID

    schedule_df['Job_ID'] = [str(i + 1) for i in range(I) for s in range(S)]
    color_list = ["#C0392B", "#EC7063", "#1F618D", "#34495E", "#3498DB", "#1E8449", "#28B463", "#F1C40F", "#F39C12", "#A6ACAF", "#7D3C98", "#9B59B6", "#2C3E50", "#EF553B", "#636EFA", "#BDC3C7", "#F39C12"]
    fig = px.timeline(schedule_df, x_start="START_TIME", x_end="END_TIME", y="Machine_ID", color="Job_ID",
                      color_discrete_sequence=color_list[0:I],
                      category_orders={"Job_ID": [str(i+1) for i in range(I)], "Machine_ID": [str(m+1) for m in range(M)]},
                      title="Instance " + str(instance_no) + " Problem " + str(problem_no) + " Gantt Chart")
    fig.update_xaxes(tickformat="%H:%M")
    fig.update_layout(legend_title="Lot ID")
    fig.show()
    fig.write_image("Instance " + str(instance_no) + " Problem " + str(problem_no) + ".pdf")

### Instance no

In [169]:
instance_no = 1

### 1.

### Constant Data

In [219]:
order_num = [12, 11, 10]
I = order_num[instance_no - 1]
M = 4
orders = range(0, I)
machines = range(0, M)

In [220]:
# process time for order i
process_time = [[4,3,2.6,1.2,1.8,2.5,3.4,2.2,1.5,1.5,4.4,2],
                [4.2,3.9,3.7,3.6,2.7,2.7,2.9,2.2,2.6,4.4,4],
                [4.5,5,6.4,3.6,3.7,3.3,3.2,1.3,4,3.9]]
P = process_time[instance_no - 1]

# deadline for order i
deadline = [[5, 5, 5, 5, 5, 5, 5, 10, 10, 10, 10, 10],
            [10, 5, 5, 5, 5, 5, 5, 10, 10, 10, 10],
            [5, 5, 5, 5, 5, 5, 10, 10, 10, 10]]
D = deadline[instance_no - 1]

### Decision variable – Delay

In [173]:
Q1_delay = Model('Q1-delay')

# x_i is the finish time of order i
x = {}
for i in orders:
    x[i] = Q1_delay.addVar(lb = 0, vtype = GRB.CONTINUOUS, name = "x_" + str(i))

# y_im = 1 if order i process on machine m otherwise 0
y = {}
for i in orders:
    y[i] = {}
    for m in machines:
        y[i][m] = Q1_delay.addVar(lb = 0, vtype = GRB.BINARY, name = "y_" + str(i) + "," + str(m))
        
# z_ij = 1 if order i process before order i process on the same machine otherwise 0
z = {}
for i in orders:
    z[i] = {}
    for j in orders:
        z[i][j] = Q1_delay.addVar(lb = 0, vtype = GRB.BINARY, name = "z_" + str(i) + "," + str(j))
        
# l_i = 1 if order i is late otherwise 0
l = {}
for i in orders:
    l[i] = Q1_delay.addVar(lb = 0, vtype = GRB.BINARY, name = "l_" + str(i))

Q1_delay.update()

### Setting the objective function

In [174]:
Q1_delay.setObjective(quicksum(l[i] for i in orders), GRB.MINIMIZE)

### Add constraints

In [175]:
Q1_delay.addConstrs((x[j] + P[i] - x[i] <= (quicksum(P[i] for i in orders)+P[i]) * (z[i][j] + 2-y[i][m]-y[j][m]) for i in orders for j in range(i+1, I) for m in machines), 'constraints for x_i, z_ij')
Q1_delay.addConstrs((x[i] + P[j] - x[j] <= (quicksum(P[i] for i in orders)+P[j]) * (1-z[i][j] + 2-y[i][m]-y[j][m]) for i in orders for j in range(i+1, I) for m in machines), 'constraints for x_i, z_ij')
Q1_delay.addConstrs((x[i] <= D[i] + quicksum(P[i] for i in orders) * l[i] for i in orders), 'constraints for l_i')
Q1_delay.addConstrs((quicksum(y[i][m] for m in machines) == 1 for i in orders), 'allocate orders to machines')
Q1_delay.addConstrs((x[i] >= P[i] for i in orders), 'minimum x_i')

Q1_delay.update()

In [176]:
Q1_delay.optimize()

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (mac64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 390 rows, 160 columns and 1870 nonzeros
Model fingerprint: 0x9a4a8253
Variable types: 10 continuous, 150 integer (150 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+02]
Found heuristic solution: objective 4.0000000
Presolve removed 11 rows and 56 columns
Presolve time: 0.00s
Presolved: 379 rows, 104 columns, 1858 nonzeros
Variable types: 10 continuous, 94 integer (94 binary)

Root relaxation: objective 1.000000e+00, 53 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    1.00000    0    2    4.00000    1.00000  75.0%     -    0s
H    0     0                       2.0000000    1.00000

In [177]:
a = Q1_delay.objVal
print("objective value =", a)

objective value = 2.0


### Decision variable – makespan

In [178]:
Q1_makespan = Model('Q1-makespan')

# x_i is the finish time of order i
x = {}
for i in orders:
    x[i] = Q1_makespan.addVar(lb = 0, vtype = GRB.CONTINUOUS, name = "x_" + str(i))

# y_im = 1 if order i process on machine m otherwise 0
y = {}
for i in orders:
    y[i] = {}
    for m in machines:
        y[i][m] = Q1_makespan.addVar(lb = 0, vtype = GRB.BINARY, name = "y_" + str(i) + "," + str(m))
        
# z_ij = 1 if order i process before order j and i, j process on the same machine otherwise 0
z = {}
for i in orders:
    z[i] = {}
    for j in orders:
        z[i][j] = Q1_makespan.addVar(lb = 0, vtype = GRB.BINARY, name = "z_" + str(i) + "," + str(j))
        
# l_i = 1 if order i is late otherwise 0
l = {}
for i in orders:
    l[i] = Q1_makespan.addVar(lb = 0, vtype = GRB.BINARY, name = "l_" + str(i))

# w is the makespan of the solution
w = Q1_makespan.addVar(lb = 0, vtype = GRB.CONTINUOUS, name = "w")

Q1_makespan.update()

### Setting the objective function

In [179]:
Q1_makespan.setObjective(w, GRB.MINIMIZE)

### Add constraints

In [180]:
Q1_makespan.addConstrs((x[j] + P[i] - x[i] <= (quicksum(P[i] for i in orders)+P[i]) * (z[i][j] + 2-y[i][m]-y[j][m]) for i in orders for j in range(i+1, I) for m in machines), 'constraints for x_i, z_ij')
Q1_makespan.addConstrs((x[i] + P[j] - x[j] <= (quicksum(P[i] for i in orders)+P[i]) * (1-z[i][j] + 2-y[i][m]-y[j][m]) for i in orders for j in range(i+1, I) for m in machines), 'constraints for x_i, z_ij')
Q1_makespan.addConstrs((x[i] <= D[i] + quicksum(P[i] for i in orders) * l[i] for i in orders), 'constraints for l_i')
Q1_makespan.addConstrs((quicksum(y[i][m] for m in machines) == 1 for i in orders), 'allocate orders to machines')
Q1_makespan.addConstrs((x[i] >= P[i] for i in orders), 'minimum x_i')
Q1_makespan.addConstrs((w >= x[i] for i in orders), 'makespan')
Q1_makespan.addConstr((quicksum(l[i] for i in orders) == a), 'delay num')

Q1_makespan.update()

In [181]:
Q1_makespan.optimize()

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (mac64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 401 rows, 161 columns and 1900 nonzeros
Model fingerprint: 0x8c8fec77
Variable types: 11 continuous, 150 integer (150 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+02]
Presolve removed 12 rows and 57 columns
Presolve time: 0.00s
Presolved: 389 rows, 104 columns, 1885 nonzeros
Variable types: 11 continuous, 93 integer (93 binary)

Root relaxation: objective 6.400000e+00, 53 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    6.40000    0    9          -    6.40000      -     -    0s
H    0     0                      11.4000000    6.40000  43.9%     -    0s
     0     0    6.40000   

In [182]:
print("objective value =", Q1_makespan.objVal)

objective value = 10.099999998890004


## Results

In [183]:
x_opt = {}
for i in orders:
    x_opt[i] = {}

y_opt = {}
for i in orders:
    y_opt[i] = {}
    for m in machines:
        y_opt[i][m] = {}

for var in Q1_makespan.getVars():
    index = [i for i in var.varName[2:].split(',')]
    if var.varName[0] == 'x':
        x_opt[int(index[0])] = var.x
    elif var.varName[0] == 'y':
        y_opt[int(index[0])][int(index[1])] = var.x

## Plot Gantt Charts

In [184]:
gantt_plot_1(x_opt, P, y_opt, instance_no)

### 2.

### Constant Data

In [185]:
order_num = [12, 11, 10]
I = order_num[instance_no - 1]
M = 4 # machine num
S = 2 # stage num
orders = range(0, I)
machines = range(0, M)
stages = range(0, S)

In [186]:
# process time for order i at stage s
process_time = [[[2.7, 1.3],
                 [1.6, 1.4],
                 [0.7, 1.9],
                 [0.5, 0.7],
                 [0.8, 1],
                 [2.5, 0],
                 [1.4, 2],
                 [1.1, 1.1],
                 [0.8, 0.7],
                 [1, 0.5],
                 [3, 1.4],
                 [2, 0]],
                [[2.7, 1.5],
                 [1.6, 2.3],
                 [1, 2.7],
                 [2.8, 0.8],
                 [0.8, 1.9],
                 [2.7, 0],
                 [1.4, 1.5],
                 [2.2, 0],
                 [0.8, 1.8],
                 [2.2, 2.2],
                 [2.5, 1.5]],
                [[3, 1.5],
                 [3, 2],
                 [3, 3.4],
                 [3, 0.6],
                 [1.2, 2.5],
                 [3.3, 0],
                 [1.7, 1.5],
                 [1.3, 0],
                 [1.8, 2.2],
                 [2.6, 1.3]]]

P = process_time[instance_no - 1]

### Decision variable – Delay

In [187]:
Q2_delay = Model('Q2-delay')

# x_is is the finish time of order i at stage s
x = {}
for i in orders:
    x[i] = {}
    for s in stages:
        x[i][s] = Q2_delay.addVar(lb = 0, vtype = GRB.CONTINUOUS, name = "x_" + str(i) + "," + str(s))

# y_ism = 1 if order i process on machine m  at stage s otherwise 0
y = {}
for i in orders:
    y[i] = {}
    for s in stages:
        y[i][s] = {}
        for m in machines:
            y[i][s][m] = Q2_delay.addVar(lb = 0, vtype = GRB.BINARY, name = "y_" + str(i) + "," + str(s) + "," + str(m))
        
# z_isjt = 1 if order i's stage s process before order j's stage t and they both process on the same machine otherwise 0
z = {}
for i in orders:
    z[i] = {}
    for s in stages:
        z[i][s] = {}
        for j in orders:
            z[i][s][j] = {}
            for t in stages:
                z[i][s][j][t] = Q2_delay.addVar(lb = 0, vtype = GRB.BINARY, name = "z_" + str(i) + "," + str(s) + "," + str(j) + "," + str(t))
        
# l_i = 1 if order i is late otherwise 0
l = {}
for i in orders:
    l[i] = Q2_delay.addVar(lb = 0, vtype = GRB.BINARY, name = "l_" + str(i))

Q2_delay.update()

### Setting the objective function

In [188]:
Q2_delay.setObjective(quicksum(l[i] for i in orders), GRB.MINIMIZE)

### Add constraints

In [189]:
Q2_delay.addConstrs((x[j][t] + P[i][s] - x[i][s] <= (quicksum(P[i][s] for i in orders for s in stages)+P[i][s]) * (z[i][s][j][t] + 2-y[i][s][m]-y[j][t][m]) for i in orders for j in range(i+1, I) for m in machines for s in stages for t in stages), 'constraints for x_is, z_isjt')
Q2_delay.addConstrs((x[i][s] + P[j][t] - x[j][t] <= (quicksum(P[i][s] for i in orders for s in stages)+P[j][t]) * (1-z[i][s][j][t] + 2-y[i][s][m]-y[j][t][m]) for i in orders for j in range(i+1, I) for m in machines for s in stages for t in stages), 'constraints for x_is, z_isjt')
Q2_delay.addConstrs((x[i][1] <= D[i] + quicksum(P[i][s] for s in stages for i in orders) * l[i] for i in orders), 'constraints for l_i')
Q2_delay.addConstrs((quicksum(y[i][s][m] for m in machines) == 1 for i in orders for s in stages), 'allocate orders to machines')
Q2_delay.addConstrs((x[i][s] >= P[i][s] for i in orders for s in stages), 'minimum x_i')
Q2_delay.addConstrs((x[i][1] >= x[i][0] + P[i][1] for i in orders), 'stage 2 should start after stage 1 is finished')

Q2_delay.update()

In [190]:
Q2_delay.optimize()

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (mac64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 1500 rows, 510 columns and 7340 nonzeros
Model fingerprint: 0x27f846aa
Variable types: 20 continuous, 490 integer (490 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [6e-01, 1e+02]
Found heuristic solution: objective 5.0000000
Presolve removed 21 rows and 221 columns
Presolve time: 0.01s
Presolved: 1479 rows, 289 columns, 7346 nonzeros
Variable types: 20 continuous, 269 integer (269 binary)

Root relaxation: objective 1.000000e+00, 161 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    1.00000    0    1    5.00000    1.00000  80.0%     -    0s
H    0     0                       3.0000000    1

In [191]:
a = Q2_delay.objVal
print("objective value =", a)

objective value = 2.0


### Decision variable – makespan

In [192]:
Q2_makespan = Model('Q2-makespan')

# x_is is the finish time of order i at stage s
x = {}
for i in orders:
    x[i] = {}
    for s in stages:
        x[i][s] = Q2_makespan.addVar(lb = 0, vtype = GRB.CONTINUOUS, name = "x_" + str(i) + "," + str(s))

# y_ism = 1 if order i process on machine m  at stage s otherwise 0
y = {}
for i in orders:
    y[i] = {}
    for s in stages:
        y[i][s] = {}
        for m in machines:
            y[i][s][m] = Q2_makespan.addVar(lb = 0, vtype = GRB.BINARY, name = "y_" + str(i) + "," + str(s) + "," + str(m))
        
# z_isjt = 1 if order i's stage s process before order j's stage t and they both process on the same machine otherwise 0
z = {}
for i in orders:
    z[i] = {}
    for s in stages:
        z[i][s] = {}
        for j in orders:
            z[i][s][j] = {}
            for t in stages:
                z[i][s][j][t] = Q2_makespan.addVar(lb = 0, vtype = GRB.BINARY, name = "z_" + str(i) + "," + str(s) + "," + str(j) + "," + str(t))
        
# l_i = 1 if order i is late otherwise 0
l = {}
for i in orders:
    l[i] = Q2_makespan.addVar(lb = 0, vtype = GRB.BINARY, name = "l_" + str(i))

# w is the makespan of the solution
w = Q2_makespan.addVar(lb = 0, vtype = GRB.CONTINUOUS, name = "w")

Q2_makespan.update()

### Setting the objective function

In [193]:
Q2_makespan.setObjective(w, GRB.MINIMIZE)

### Add constraints

In [194]:
Q2_makespan.addConstrs((x[j][t] + P[i][s] - x[i][s] <= (quicksum(P[i][s] for i in orders for s in stages)+P[i][s]) * (z[i][s][j][t] + 2-y[i][s][m]-y[j][t][m]) for i in orders for j in range(i+1, I) for m in machines for s in stages for t in stages), 'constraints for x_is, z_isjt')
Q2_makespan.addConstrs((x[i][s] + P[j][t] - x[j][t] <= (quicksum(P[i][s] for i in orders for s in stages)+P[j][t]) * (1-z[i][s][j][t] + 2-y[i][s][m]-y[j][t][m]) for i in orders for j in range(i+1, I) for m in machines for s in stages for t in stages), 'constraints for x_is, z_isjt')
Q2_makespan.addConstrs((x[i][1] <= D[i] + quicksum(P[i][s] for s in stages for i in orders) * l[i] for i in orders), 'constraints for l_i')
Q2_makespan.addConstrs((quicksum(y[i][s][m] for m in machines) == 1 for i in orders for s in stages), 'allocate orders to machines')
Q2_makespan.addConstrs((x[i][s] >= P[i][s] for i in orders for s in stages), 'minimum x_i')
Q2_makespan.addConstrs((x[i][1] >= x[i][0] + P[i][1] for i in orders), 'stage 2 should start after stage 1 is finished')
Q2_makespan.addConstrs((w >= x[i][1] for i in orders), 'makespan')
Q2_makespan.addConstr((quicksum(l[i] for i in orders) == a), 'delay num')

Q2_makespan.update()

In [195]:
Q2_makespan.optimize()

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (mac64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 1511 rows, 511 columns and 7370 nonzeros
Model fingerprint: 0x7e27c2d4
Variable types: 21 continuous, 490 integer (490 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [6e-01, 1e+02]
Presolve removed 22 rows and 222 columns
Presolve time: 0.02s
Presolved: 1489 rows, 289 columns, 7345 nonzeros
Variable types: 21 continuous, 268 integer (268 binary)

Root relaxation: objective 6.400000e+00, 161 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    6.40000    0   12          -    6.40000      -     -    0s
     0     0    6.40000    0   22          -    6.40000      -     -    0s
     0     0    6.40

In [196]:
print("objective value =", Q2_makespan.objVal)

objective value = 9.800000000000008


## Results

In [197]:
x_opt = {}
for i in orders:
    x_opt[i] = {}
    for s in stages:
        x_opt[i][s] = {}

y_opt = {}
for i in orders:
    y_opt[i] = {}
    for s in stages:
        y_opt[i][s] = {}
        for m in machines:
            y_opt[i][s][m] = {}

for var in Q2_makespan.getVars():
    index = [i for i in var.varName[2:].split(',')]
    if var.varName[0] == 'x':
        x_opt[int(index[0])][int(index[1])] = var.x
    elif var.varName[0] == 'y':
        y_opt[int(index[0])][int(index[1])][int(index[2])] = var.x

## Plot Gantt Charts

In [198]:
gantt_plot_2_3(x_opt, P, y_opt, instance_no, 2)

### 3.

### Constant Data

In [221]:
order_num = [12, 11, 10]
I = order_num[instance_no - 1] # order num
M = 5 # machine num
S = 2 # stage num
M_C = 1
orders = range(0, I)
machines = range(0, M)
stages = range(0, S)
cook_machines = range(0, M_C)

In [222]:
# process time for order i at stage s
process_time = [[[2.7, 1.3],
                 [1.6, 1.4],
                 [0.7, 1.9],
                 [0.5, 0.7],
                 [0.8, 1],
                 [2.5, 0],
                 [1.4, 2],
                 [1.1, 1.1],
                 [0.8, 0.7],
                 [1, 0.5],
                 [3, 1.4],
                 [2, 0]],
                [[2.7, 1.5],
                 [1.6, 2.3],
                 [1, 2.7],
                 [2.8, 0.8],
                 [0.8, 1.9],
                 [2.7, 0],
                 [1.4, 1.5],
                 [2.2, 0],
                 [0.8, 1.8],
                 [2.2, 2.2],
                 [2.5, 1.5]],
                [[3, 1.5],
                 [3, 2],
                 [3, 3.4],
                 [3, 0.6],
                 [1.2, 2.5],
                 [3.3, 0],
                 [1.7, 1.5],
                 [1.3, 0],
                 [1.8, 2.2],
                 [2.6, 1.3]]]

P = process_time[instance_no - 1]

# thers's only cook for order i at stage s
onlycook = [[[1, 0],
             [0, 1],
             [1, 0],
             [0, 1],
             [1, 0],
             [1, 0],
             [0, 0],
             [0, 0],
             [1, 0],
             [0, 1],
             [1, 0],
             [0, 0]],
            [[1, 0],
             [1, 0],
             [1, 0],
             [0, 0],
             [1, 0],
             [1, 0],
             [0, 0],
             [0, 0],
             [1, 0],
             [0, 0],
             [0, 1]],
            [[0, 0],
             [0, 1],
             [1, 0],
             [0, 1],
             [1, 0],
             [1, 0],
             [0, 0],
             [0, 0],
             [0, 1],
             [0, 0]]]

C = onlycook[instance_no - 1]

### Decision variable – Delay

In [223]:
Q3_delay = Model('Q3-delay')

# x_is is the finish time of order i at stage s
x = {}
for i in orders:
    x[i] = {}
    for s in stages:
        x[i][s] = Q3_delay.addVar(lb = 0, vtype = GRB.CONTINUOUS, name = "x_" + str(i) + "," + str(s))

# y_ism = 1 if order i process on machine m  at stage s otherwise 0
y = {}
for i in orders:
    y[i] = {}
    for s in stages:
        y[i][s] = {}
        for m in machines:
            y[i][s][m] = Q3_delay.addVar(lb = 0, vtype = GRB.BINARY, name = "y_" + str(i) + "," + str(s) + "," + str(m))
        
# z_isjt = 1 if order i's stage s process before order j's stage t and they both process on the same machine otherwise 0
z = {}
for i in orders:
    z[i] = {}
    for s in stages:
        z[i][s] = {}
        for j in orders:
            z[i][s][j] = {}
            for t in stages:
                z[i][s][j][t] = Q3_delay.addVar(lb = 0, vtype = GRB.BINARY, name = "z_" + str(i) + "," + str(s) + "," + str(j) + "," + str(t))
        
# l_i = 1 if order i is late otherwise 0
l = {}
for i in orders:
    l[i] = Q3_delay.addVar(lb = 0, vtype = GRB.BINARY, name = "l_" + str(i))

Q3_delay.update()

### Setting the objective function

In [224]:
Q3_delay.setObjective(quicksum(l[i] for i in orders), GRB.MINIMIZE)

### Add constraints

In [225]:
Q3_delay.addConstrs((x[j][t] + P[i][s] - x[i][s] <= (quicksum(P[i][s] for i in orders for s in stages)+P[i][s]) * (z[i][s][j][t] + 2-y[i][s][m]-y[j][t][m]) for i in orders for j in range(i+1, I) for m in machines for s in stages for t in stages), 'constraints for x_is, z_isjt')
Q3_delay.addConstrs((x[i][s] + P[j][t] - x[j][t] <= (quicksum(P[i][s] for i in orders for s in stages)+P[j][t]) * (1-z[i][s][j][t] + 2-y[i][s][m]-y[j][t][m]) for i in orders for j in range(i+1, I) for m in machines for s in stages for t in stages), 'constraints for x_is, z_isjt')
Q3_delay.addConstrs((x[i][1] <= D[i] + quicksum(P[i][s] for s in stages for i in orders) * l[i] for i in orders), 'constraints for l_i')
Q3_delay.addConstrs((quicksum(y[i][s][m] for m in machines) == 1 for i in orders for s in stages), 'allocate orders to machines')
Q3_delay.addConstrs((x[i][s] >= P[i][s] for i in orders for s in stages), 'minimum x_i')
Q3_delay.addConstrs((x[i][1] >= x[i][0] + P[i][1] for i in orders), 'stage 2 should start after stage 1 is finished')
Q3_delay.addConstrs((y[i][s][m] <= C[i][s] for i in orders for s in stages for m in cook_machines), 'constraints for cook machines')

Q3_delay.update()

In [226]:
Q3_delay.optimize()

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (mac64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 2736 rows, 732 columns and 13416 nonzeros
Model fingerprint: 0xedbcdcfa
Variable types: 24 continuous, 708 integer (708 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e-01, 1e+02]
Found heuristic solution: objective 6.0000000
Presolve removed 260 rows and 327 columns
Presolve time: 0.02s
Presolved: 2476 rows, 405 columns, 12293 nonzeros
Variable types: 24 continuous, 381 integer (381 binary)

Root relaxation: objective 0.000000e+00, 132 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

H    0     0                       0.0000000    0.00000  0.00%     -    0s
     0     0    0.00000    0    1    0.00000  

In [227]:
a = Q3_delay.objVal
print("objective value =", a)

objective value = 0.0


### Decision variable – makespan

In [228]:
Q3_makespan = Model('Q3-makespan')

# x_is is the finish time of order i at stage s
x = {}
for i in orders:
    x[i] = {}
    for s in stages:
        x[i][s] = Q3_makespan.addVar(lb = 0, vtype = GRB.CONTINUOUS, name = "x_" + str(i) + "," + str(s))

# y_ism = 1 if order i process on machine m  at stage s otherwise 0
y = {}
for i in orders:
    y[i] = {}
    for s in stages:
        y[i][s] = {}
        for m in machines:
            y[i][s][m] = Q3_makespan.addVar(lb = 0, vtype = GRB.BINARY, name = "y_" + str(i) + "," + str(s) + "," + str(m))
        
# z_isjt = 1 if order i's stage s process before order j's stage t and they both process on the same machine otherwise 0
z = {}
for i in orders:
    z[i] = {}
    for s in stages:
        z[i][s] = {}
        for j in orders:
            z[i][s][j] = {}
            for t in stages:
                z[i][s][j][t] = Q3_makespan.addVar(lb = 0, vtype = GRB.BINARY, name = "z_" + str(i) + "," + str(s) + "," + str(j) + "," + str(t))
        
# l_i = 1 if order i is late otherwise 0
l = {}
for i in orders:
    l[i] = Q3_makespan.addVar(lb = 0, vtype = GRB.BINARY, name = "l_" + str(i))

# w is the makespan of the solution
w = Q3_makespan.addVar(lb = 0, vtype = GRB.CONTINUOUS, name = "w")

Q3_makespan.update()

### Setting the objective function

In [229]:
Q3_makespan.setObjective(w, GRB.MINIMIZE)

### Add constraints

In [230]:
Q3_makespan.addConstrs((x[j][t] + P[i][s] - x[i][s] <= (quicksum(P[i][s] for i in orders for s in stages)+P[i][s]) * (z[i][s][j][t] + 2-y[i][s][m]-y[j][t][m]) for i in orders for j in range(i+1, I) for m in machines for s in stages for t in stages), 'constraints for x_is, z_isjt')
Q3_makespan.addConstrs((x[i][s] + P[j][t] - x[j][t] <= (quicksum(P[i][s] for i in orders for s in stages)+P[j][t]) * (1-z[i][s][j][t] + 2-y[i][s][m]-y[j][t][m]) for i in orders for j in range(i+1, I) for m in machines for s in stages for t in stages), 'constraints for x_is, z_isjt')
Q3_makespan.addConstrs((x[i][1] <= D[i] + quicksum(P[i][s] for s in stages for i in orders) * l[i] for i in orders), 'constraints for l_i')
Q3_makespan.addConstrs((quicksum(y[i][s][m] for m in machines) == 1 for i in orders for s in stages), 'allocate orders to machines')
Q3_makespan.addConstrs((x[i][s] >= P[i][s] for i in orders for s in stages), 'minimum x_i')
Q3_makespan.addConstrs((x[i][1] >= x[i][0] + P[i][1] for i in orders), 'stage 2 should start after stage 1 is finished')
Q3_makespan.addConstrs((y[i][s][m] <= C[i][s] for i in orders for s in stages for m in cook_machines), 'constraints for cook machines')
Q3_makespan.addConstrs((w >= x[i][1] for i in orders), 'makespan')
Q3_makespan.addConstr((quicksum(l[i] for i in orders) == a), 'delay num')
# Q3_makespan.addConstr((w == 6.1), 'delay num')

Q3_makespan.update()

In [231]:
Q3_makespan.optimize()

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (mac64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 2749 rows, 733 columns and 13452 nonzeros
Model fingerprint: 0x61b948ca
Variable types: 25 continuous, 708 integer (708 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e-01, 1e+02]
Presolve removed 525 rows and 340 columns
Presolve time: 0.03s
Presolved: 2224 rows, 393 columns, 11033 nonzeros
Variable types: 25 continuous, 368 integer (368 binary)

Root relaxation: objective 4.400000e+00, 134 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    4.40000    0   10          -    4.40000      -     -    0s
H    0     0                       9.4000000    4.40000  53.2%     -    0s
     0     0    4

In [210]:
print("objective value =", Q3_makespan.objVal)

objective value = 9.399999999999997


## Results

In [211]:
x_opt = {}
for i in orders:
    x_opt[i] = {}
    for s in stages:
        x_opt[i][s] = {}

y_opt = {}
for i in orders:
    y_opt[i] = {}
    for s in stages:
        y_opt[i][s] = {}
        for m in machines:
            y_opt[i][s][m] = {}

for var in Q3_makespan.getVars():
    index = [i for i in var.varName[2:].split(',')]
    if var.varName[0] == 'x':
        x_opt[int(index[0])][int(index[1])] = var.x
    elif var.varName[0] == 'y':
        y_opt[int(index[0])][int(index[1])][int(index[2])] = var.x

## Plot Gantt Charts

In [212]:
gantt_plot_2_3(x_opt, P, y_opt, instance_no, 3)